# Historical Portfolio Construction
#### Northeastern University Student Value Fund
#### Miles Child - Portfolio Manager
____________

In this notebook, we will conduct a historical analysis of fund performance from inception to present (Sept 2023). This will be done with the purpose of:

- Understanding our performance and risk/return metrics against various benchmarks
- Illustrating the negative impact of haphazard timing/trade execution on portfolio performance
- Demonstrating how a theoretical portfolio with rational holdings, weights, and execution times would have performed during the same period

____________

In [1]:
# imports
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import yfinance as yf  # <-- for hist px data
from portfolio_builder import Trade, PortfolioBuilder
import datetime

Ingesting historical portfolio data

In [2]:
transactions = pd.read_excel('../data/Investment_Transaction_Detail (1).xlsx')
transactions.head()

,Consolidation,Account Number,From date,Through date,I-TRAN,C-TRD-STAT-7,Value,Quantity,Currency code,Date,...,N-INV-SUB-CATG,Error code,Ticker,N-CNTRY-P2,Consolidation Audit Indicator Flag,Accrued Interest,C-ASSET-LIAB,Narrative - Short,Mutual Fund Flag,Wash sale adjustment
0,4475530,4475530,2009-01-01,2023-09-08,647038213,Settled,200000.00,200000.00,USD,2019-01-09,...,Cash,,CASH,United States,N,0,,NaN,,0
1,4475530,4475530,2009-01-01,2023-09-08,647038213,Settled,3847.50,25.00,USD,2019-01-09,...,Common Stock,,PH,United States,N,0,,PURCHASED 25.00 SHARES 01-07-19 AT A PRICE OF ...,,0
2,4475530,4475530,2009-01-01,2023-09-08,647038213,Settled,-3847.50,-3874.50,USD,2019-01-09,...,Cash,,CASH,United States,N,0,,NaN,,0
3,4475530,4475530,2009-01-01,2023-09-08,573190268,Settled,6208.75,725.00,USD,2019-04-26,...,Common Stock,,NTDOY,Japan,N,0,,PURCHASED 145.00 SHARES 04-24-19 AT A PRICE OF...,,0
4,4475530,4475530,2009-01-01,2023-09-08,647038213,Settled,-6208.75,-6208.75,USD,2019-04-26,...,Cash,,CASH,United States,N,0,,NaN,,0


In [3]:
trades = []
for i in range(len(transactions)):
    trade = Trade(ticker=transactions['Ticker'][i],
                  date=transactions['Date'][i],
                  qty=transactions['Quantity'][i],
                  price=transactions['Value'][i] / transactions['Quantity'][i])
    trades.append(trade)
trades

[CASH 200000.0 2019-01-09 00:00:00 1,
 PH 25.0 2019-01-09 00:00:00 153.9,
 CASH -3874.5 2019-01-09 00:00:00 1,
 NTDOY 725.0 2019-04-26 00:00:00 8.563793103448276,
 CASH -6208.75 2019-04-26 00:00:00 1,
 ABMD 32.0 2019-04-26 00:00:00 260.3725,
 CASH -8331.92 2019-04-26 00:00:00 1,
 HCA -35.0 2019-04-26 00:00:00 80.69657142857143,
 CASH 2824.38 2019-04-26 00:00:00 1,
 CSIQ 350.0 2019-10-31 00:00:00 17.779,
 CASH -6222.65 2019-10-31 00:00:00 1,
 NTDOY 570.0 2019-12-06 00:00:00 10.205543859649122,
 CASH -5817.16 2019-12-06 00:00:00 1,
 SATS 298.0 2019-12-06 00:00:00 40.0701677852349,
 CASH -11940.91 2019-12-06 00:00:00 1,
 SCPL 1062.0 2019-12-06 00:00:00 12.789096045197741,
 CASH -13582.02 2019-12-06 00:00:00 1,
 EVVTY 190.0 2021-04-15 00:00:00 166.04263157894735,
 CASH -31548.1 2021-04-15 00:00:00 1,
 REGI 199.0 2021-04-15 00:00:00 60.39015075376884,
 CASH -12017.64 2021-04-15 00:00:00 1,
 GLXZ 11299.0 2021-04-15 00:00:00 3.005799628285689,
 CASH -33962.53 2021-04-15 00:00:00 1,
 PCYO 1725

In [4]:
backup_data = pd.read_excel('../data/backup_data.xlsx')
backup_data.head()

,Date,ABMD,MAXR,ECOM,REGI
0,2022-11-12,374.10,52.99,23.09,61.50
1,2022-11-11,373.99,52.99,23.08,61.45
2,2022-11-10,372.78,52.99,23.08,61.47
3,2022-11-09,374.00,52.99,23.06,61.37
4,2022-11-08,374.00,52.99,23.06,61.39


In [5]:
dt = datetime.datetime(2022, 11, 12)
px = backup_data[backup_data['Date'] <= dt]['MAXR'].values[0]
px

52.99

In [6]:
pb = PortfolioBuilder(trades, backup_price_data=backup_data)
transactions = pb.get_transactions()
transactions.head()

,ticker,qty,price,date
0,CASH,200000.00,1.000000,2019-01-09
0,PH,25.00,153.900000,2019-01-09
0,CASH,-3874.50,1.000000,2019-01-09
0,NTDOY,725.00,8.563793,2019-04-26
0,CASH,-6208.75,1.000000,2019-04-26


In [7]:
portfolio = pb.get_portfolio()
portfolio.head(25)

,Ticker,Weight,Value,Qty,Avg,Current,Gain/Loss $,Gain/Loss %
0,CASH,0.224039,46815.81,46815.81,1.000000,1,0.000000,0.000000
0,NTDOY,0.089763,18757.200365,1595.00,9.796683,11.76,3131.490365,0.200406
0,EVVTY,0.085682,17904.249573,175.00,166.042629,102.309998,-11153.210427,-0.383833
0,CVLT,0.093051,19444.25,271.00,68.409483,71.75,905.280000,0.048831
0,NSIT,0.102282,21372.999115,145.00,104.323931,147.399994,6246.029115,0.412907
0,DAR,0.066034,13798.560394,323.00,58.640650,42.720001,-5142.369606,-0.271495
0,SMLP,0.115544,24144.300442,1158.00,15.870898,20.85,5765.800442,0.313725
0,PYPL,0.067269,14056.640076,248.00,74.277218,56.68,-4364.109924,-0.236913
0,GM,0.075573,15792.000427,560.00,31.839607,28.200001,-2038.179573,-0.114311
0,MTN,0.080763,16876.50032,75.00,239.853333,225.020004,-1112.499680,-0.061843


In [8]:
prices = pb.get_price_df(ticker="MAXR", start_date=datetime.datetime(2021, 1, 1), end_date=datetime.datetime(2023, 9, 1))
prices.to_excel('prices.xlsx')
prices.head(25)

,Date,MAXR
2021-01-01,NaT,NaN
2021-01-02,NaT,NaN
2021-01-03,NaT,NaN
2021-01-04,NaT,NaN
2021-01-05,NaT,NaN
2021-01-06,NaT,NaN
2021-01-07,NaT,NaN
2021-01-08,NaT,NaN
2021-01-09,NaT,NaN
2021-01-10,NaT,NaN


In [9]:
df = pb.backup_price_data

start_date = datetime.datetime(2020, 1, 1)
end_date = datetime.datetime(2022, 11, 12)
ticker = "MAXR"
price_df = pd.DataFrame(index=[d.date() for d in pd.date_range(start=start_date, end=end_date)])
        
if ticker in df.columns:
    tmp_price_df = df[df['Date'] <= end_date].drop(columns=[c for c in df.columns if c != ticker and c != 'Date'])
else:
    tmp_price_df = yf.Ticker(ticker).history(period='1d', start=start_date, end=end_date)
    tmp_price_df.index = tmp_price_df.index.date
    tmp_price_df.drop(columns=[c for c in tmp_price_df.columns if c != 'Close'], inplace=True)

# merge on the dates that we have
price_df = price_df.merge(tmp_price_df, how='left', left_index=True, right_index=True)
# fill forward
price_df = price_df.fillna(method='ffill')
# fill backward
price_df = price_df.fillna(method='bfill')
tmp_price_df.head()

,Date,MAXR
0,2022-11-12,52.99
1,2022-11-11,52.99
2,2022-11-10,52.99
3,2022-11-09,52.99
4,2022-11-08,52.99


In [10]:
vot = pb.get_value_over_time(start_date=datetime.datetime(2020, 1, 1), end_date=datetime.datetime(2023, 9, 12))
vot.to_excel('vot.xlsx')

AttributeError: 'NotImplementedType' object has no attribute '_indexed_same'

In [10]:
vot.head()

,Value
2020-01-01,NaN
2020-01-02,NaN
2020-01-03,NaN
2020-01-04,NaN
2020-01-05,NaN


________

In [16]:
dated_portfolio = pb.get_portfolio(up_to=datetime.datetime(2022, 12, 31))
dated_portfolio.head(25)

MAXR: No data found, symbol may be delisted


,Ticker,Weight,Value,Qty,Avg,Current,Gain/Loss $,Gain/Loss %
0,CASH,0.278592,48427.21,48427.21,1.000000,1,0.000000,0.000000
0,NTDOY,0.096345,16747.5,1595.00,9.796683,10.5,1121.790000,0.071791
0,EVVTY,0.099841,17355.191612,175.00,166.042629,99.172523,-11702.268388,-0.402729
0,GLXZ,0.067704,11768.829615,5051.00,3.005799,2.33,-3413.460385,-0.224832
0,FREE,0.039623,6887.560257,1684.00,13.572298,4.09,-15968.189743,-0.698651
0,CVLT,0.099636,17319.609959,271.00,68.409483,63.91,-1219.360041,-0.065773
0,MITK,0.066026,11477.220045,1182.00,15.732800,9.71,-7118.949955,-0.382818
0,MAXR,0.136569,23739.52,448.00,30.315089,52.99,10158.360000,0.747974
0,NSIT,0.085251,14818.999557,145.00,104.323931,102.199997,-307.970443,-0.020359
0,TUP,0.030415,5286.930223,1299.00,15.190600,4.07,-14445.659777,-0.732071


In [15]:
# get weekly port values over the last year
agg_values = {}
for i in range(1, 53):
    dt = datetime.datetime(2022, 1, 1) + datetime.timedelta(days=i*7)
    print(dt)
    dated_portfolio = pb.get_portfolio(up_to=dt)
    agg_val = dated_portfolio['Value'].sum()
    agg_values[dt] = agg_val

agg_values

2022-01-08 00:00:00
Got error from yahoo api for ticker ECOM, Error: {'code': 'Not Found', 'description': 'No data found, symbol may be delisted'}
- ECOM: No timezone found, symbol may be delisted
MAXR: No data found, symbol may be delisted
2022-01-15 00:00:00
Got error from yahoo api for ticker ECOM, Error: {'code': 'Not Found', 'description': 'No data found, symbol may be delisted'}
- ECOM: No timezone found, symbol may be delisted
MAXR: No data found, symbol may be delisted
2022-01-22 00:00:00
Got error from yahoo api for ticker ECOM, Error: {'code': 'Not Found', 'description': 'No data found, symbol may be delisted'}
- ECOM: No timezone found, symbol may be delisted
MAXR: No data found, symbol may be delisted
2022-01-29 00:00:00
Got error from yahoo api for ticker ECOM, Error: {'code': 'Not Found', 'description': 'No data found, symbol may be delisted'}
- ECOM: No timezone found, symbol may be delisted
MAXR: No data found, symbol may be delisted
2022-02-05 00:00:00
Got error from y

{datetime.datetime(2022, 1, 8, 0, 0): 262644.8783816338,
 datetime.datetime(2022, 1, 15, 0, 0): 259655.82911706925,
 datetime.datetime(2022, 1, 22, 0, 0): 251678.94962841034,
 datetime.datetime(2022, 1, 29, 0, 0): 254619.38347346304,
 datetime.datetime(2022, 2, 5, 0, 0): 252035.3180265808,
 datetime.datetime(2022, 2, 12, 0, 0): 251265.54489154817,
 datetime.datetime(2022, 2, 19, 0, 0): 240076.01075115206,
 datetime.datetime(2022, 2, 26, 0, 0): 260105.16839775085,
 datetime.datetime(2022, 3, 5, 0, 0): 245410.80344646453,
 datetime.datetime(2022, 3, 12, 0, 0): 237435.0323364067,
 datetime.datetime(2022, 3, 19, 0, 0): 256577.38973911287,
 datetime.datetime(2022, 3, 26, 0, 0): 257661.02874383927,
 datetime.datetime(2022, 4, 2, 0, 0): 263116.20509296417,
 datetime.datetime(2022, 4, 9, 0, 0): 251124.95835353853,
 datetime.datetime(2022, 4, 16, 0, 0): 228276.84178684236,
 datetime.datetime(2022, 4, 23, 0, 0): 219828.6032998848,
 datetime.datetime(2022, 4, 30, 0, 0): 211543.960828228,
 datetim

In [17]:
# get weekly port values over the last year
agg_values_2 = {}
for i in range(1, 53):
    dt = datetime.datetime(2019, 1, 10) + datetime.timedelta(days=i*7)
    print(dt)
    dated_portfolio = pb.get_portfolio(up_to=dt)
    agg_val = dated_portfolio['Value'].sum()
    agg_values_2[dt] = agg_val

agg_values_2

2019-01-17 00:00:00
2019-01-24 00:00:00
2019-01-31 00:00:00
2019-02-07 00:00:00
2019-02-14 00:00:00
2019-02-21 00:00:00
2019-02-28 00:00:00
2019-03-07 00:00:00
2019-03-14 00:00:00
2019-03-21 00:00:00
2019-03-28 00:00:00
2019-04-04 00:00:00
2019-04-11 00:00:00
2019-04-18 00:00:00
2019-04-25 00:00:00
2019-05-02 00:00:00
2019-05-09 00:00:00
2019-05-16 00:00:00
2019-05-23 00:00:00
2019-05-30 00:00:00
2019-06-06 00:00:00
2019-06-13 00:00:00
2019-06-20 00:00:00
2019-06-27 00:00:00
2019-07-04 00:00:00
2019-07-11 00:00:00
2019-07-18 00:00:00
2019-07-25 00:00:00
2019-08-01 00:00:00
2019-08-08 00:00:00
2019-08-15 00:00:00
2019-08-22 00:00:00
2019-08-29 00:00:00
2019-09-05 00:00:00
2019-09-12 00:00:00
2019-09-19 00:00:00
2019-09-26 00:00:00
2019-10-03 00:00:00
2019-10-10 00:00:00
2019-10-17 00:00:00
2019-10-24 00:00:00
2019-10-31 00:00:00
2019-11-07 00:00:00
2019-11-14 00:00:00
2019-11-21 00:00:00
2019-11-28 00:00:00
2019-12-05 00:00:00
2019-12-12 00:00:00
2019-12-19 00:00:00
2019-12-26 00:00:00


{datetime.datetime(2019, 1, 17, 0, 0): 199792.2236328125,
 datetime.datetime(2019, 1, 24, 0, 0): 199758.894241333,
 datetime.datetime(2019, 1, 31, 0, 0): 199913.8186340332,
 datetime.datetime(2019, 2, 7, 0, 0): 199873.51445007324,
 datetime.datetime(2019, 2, 14, 0, 0): 200029.61415100098,
 datetime.datetime(2019, 2, 21, 0, 0): 200165.16217041016,
 datetime.datetime(2019, 2, 28, 0, 0): 200193.33332824707,
 datetime.datetime(2019, 3, 7, 0, 0): 200059.6339111328,
 datetime.datetime(2019, 3, 14, 0, 0): 200050.8589630127,
 datetime.datetime(2019, 3, 21, 0, 0): 200039.77459716797,
 datetime.datetime(2019, 3, 28, 0, 0): 200040.92854309082,
 datetime.datetime(2019, 4, 4, 0, 0): 200320.56912231445,
 datetime.datetime(2019, 4, 11, 0, 0): 200417.55436706543,
 datetime.datetime(2019, 4, 18, 0, 0): 200511.0754852295,
 datetime.datetime(2019, 4, 25, 0, 0): 200328.65170288086,
 datetime.datetime(2019, 5, 2, 0, 0): 203174.13573451995,
 datetime.datetime(2019, 5, 9, 0, 0): 202871.05589862823,
 datetime

In [18]:
# join agg values
agg_values = agg_values_2 | agg_values
agg_values

{datetime.datetime(2019, 1, 17, 0, 0): 199792.2236328125,
 datetime.datetime(2019, 1, 24, 0, 0): 199758.894241333,
 datetime.datetime(2019, 1, 31, 0, 0): 199913.8186340332,
 datetime.datetime(2019, 2, 7, 0, 0): 199873.51445007324,
 datetime.datetime(2019, 2, 14, 0, 0): 200029.61415100098,
 datetime.datetime(2019, 2, 21, 0, 0): 200165.16217041016,
 datetime.datetime(2019, 2, 28, 0, 0): 200193.33332824707,
 datetime.datetime(2019, 3, 7, 0, 0): 200059.6339111328,
 datetime.datetime(2019, 3, 14, 0, 0): 200050.8589630127,
 datetime.datetime(2019, 3, 21, 0, 0): 200039.77459716797,
 datetime.datetime(2019, 3, 28, 0, 0): 200040.92854309082,
 datetime.datetime(2019, 4, 4, 0, 0): 200320.56912231445,
 datetime.datetime(2019, 4, 11, 0, 0): 200417.55436706543,
 datetime.datetime(2019, 4, 18, 0, 0): 200511.0754852295,
 datetime.datetime(2019, 4, 25, 0, 0): 200328.65170288086,
 datetime.datetime(2019, 5, 2, 0, 0): 203174.13573451995,
 datetime.datetime(2019, 5, 9, 0, 0): 202871.05589862823,
 datetime

In [19]:
# get weekly port values over the last year
agg_values_vF = {}
for i in range(1, (53 * 4)):
    dt = datetime.datetime(2019, 1, 10) + datetime.timedelta(days=i*7)
    print(dt)
    dated_portfolio = pb.get_portfolio(up_to=dt)
    agg_val = dated_portfolio['Value'].sum()
    agg_values_vF[dt] = agg_val

agg_values_vF

2019-01-17 00:00:00
2019-01-24 00:00:00
2019-01-31 00:00:00
2019-02-07 00:00:00
2019-02-14 00:00:00
2019-02-21 00:00:00
2019-02-28 00:00:00
2019-03-07 00:00:00
2019-03-14 00:00:00
2019-03-21 00:00:00
2019-03-28 00:00:00
2019-04-04 00:00:00
2019-04-11 00:00:00
2019-04-18 00:00:00
2019-04-25 00:00:00
2019-05-02 00:00:00
2019-05-09 00:00:00
2019-05-16 00:00:00
2019-05-23 00:00:00
2019-05-30 00:00:00
2019-06-06 00:00:00
2019-06-13 00:00:00
2019-06-20 00:00:00
2019-06-27 00:00:00
2019-07-04 00:00:00
2019-07-11 00:00:00
2019-07-18 00:00:00
2019-07-25 00:00:00
2019-08-01 00:00:00
2019-08-08 00:00:00
2019-08-15 00:00:00
2019-08-22 00:00:00
2019-08-29 00:00:00
2019-09-05 00:00:00
2019-09-12 00:00:00
2019-09-19 00:00:00
2019-09-26 00:00:00
2019-10-03 00:00:00
2019-10-10 00:00:00
2019-10-17 00:00:00
2019-10-24 00:00:00
2019-10-31 00:00:00
2019-11-07 00:00:00
2019-11-14 00:00:00
2019-11-21 00:00:00
2019-11-28 00:00:00
2019-12-05 00:00:00
2019-12-12 00:00:00
2019-12-19 00:00:00
2019-12-26 00:00:00


{datetime.datetime(2019, 1, 17, 0, 0): 199792.2236328125,
 datetime.datetime(2019, 1, 24, 0, 0): 199758.894241333,
 datetime.datetime(2019, 1, 31, 0, 0): 199913.81939697266,
 datetime.datetime(2019, 2, 7, 0, 0): 199873.5136871338,
 datetime.datetime(2019, 2, 14, 0, 0): 200029.61376953125,
 datetime.datetime(2019, 2, 21, 0, 0): 200165.16255187988,
 datetime.datetime(2019, 2, 28, 0, 0): 200193.33485412598,
 datetime.datetime(2019, 3, 7, 0, 0): 200059.63314819336,
 datetime.datetime(2019, 3, 14, 0, 0): 200050.85858154297,
 datetime.datetime(2019, 3, 21, 0, 0): 200039.7734527588,
 datetime.datetime(2019, 3, 28, 0, 0): 200040.92930603027,
 datetime.datetime(2019, 4, 4, 0, 0): 200320.56950378418,
 datetime.datetime(2019, 4, 11, 0, 0): 200417.55436706543,
 datetime.datetime(2019, 4, 18, 0, 0): 200511.0754852295,
 datetime.datetime(2019, 4, 25, 0, 0): 200328.6509399414,
 datetime.datetime(2019, 5, 2, 0, 0): 203174.13726039886,
 datetime.datetime(2019, 5, 9, 0, 0): 202871.05589862823,
 datetime

In [21]:
# Plot the portfolio value over time
fig = go.Figure()
fig.add_trace(go.Scatter(x=list(agg_values_vF.keys()), y=list(agg_values_vF.values()),
                    mode='lines',
                    name='lines'))
fig.show()

In [ ]:
# Running the same thing but for the "rational" portfolio


In [43]:
transactions_rational = pd.read_excel('../data/Investment_Transaction_Detail (1) copy2.xlsx')
transactions_rational.head()

trades_rational = []
for i in range(len(transactions_rational)):
    trade = Trade(ticker=transactions_rational['Ticker'][i],
                  date=transactions_rational['Date'][i],
                  qty=transactions_rational['Quantity'][i],
                  price=transactions_rational['Value'][i] / transactions_rational['Quantity'][i])
    trades_rational.append(trade)

pb_rational = PortfolioBuilder(trades_rational, backup_price_data=backup_data)
transactions_rational = pb_rational.get_transactions()
transactions_rational.head()

,ticker,qty,price,date
0,CASH,200000.00,1.00,2019-01-09
0,PH,25.00,157.98,2019-01-09
0,CASH,-3874.50,1.00,2019-01-09
0,NTDOY,725.00,8.61,2019-04-26
0,CASH,-6242.25,1.00,2019-04-26


In [44]:
# get weekly port values over the last year
agg_values_vF_rational = {}
for i in range(1, (53 * 4)):
    dt = datetime.datetime(2019, 1, 10) + datetime.timedelta(days=i*7)
    print(dt)
    dated_portfolio = pb_rational.get_portfolio(up_to=dt)
    agg_val = dated_portfolio['Value'].sum()
    agg_values_vF_rational[dt] = agg_val

agg_values_vF_rational

2019-01-17 00:00:00
2019-01-24 00:00:00
2019-01-31 00:00:00
2019-02-07 00:00:00
2019-02-14 00:00:00
2019-02-21 00:00:00
2019-02-28 00:00:00
2019-03-07 00:00:00
2019-03-14 00:00:00
2019-03-21 00:00:00
2019-03-28 00:00:00
2019-04-04 00:00:00
2019-04-11 00:00:00
2019-04-18 00:00:00
2019-04-25 00:00:00
2019-05-02 00:00:00
2019-05-09 00:00:00
2019-05-16 00:00:00
2019-05-23 00:00:00
2019-05-30 00:00:00
2019-06-06 00:00:00
2019-06-13 00:00:00
2019-06-20 00:00:00
2019-06-27 00:00:00
2019-07-04 00:00:00
2019-07-11 00:00:00
2019-07-18 00:00:00
2019-07-25 00:00:00
2019-08-01 00:00:00
2019-08-08 00:00:00
2019-08-15 00:00:00
2019-08-22 00:00:00
2019-08-29 00:00:00
2019-09-05 00:00:00
2019-09-12 00:00:00
2019-09-19 00:00:00
2019-09-26 00:00:00
2019-10-03 00:00:00
2019-10-10 00:00:00
2019-10-17 00:00:00
2019-10-24 00:00:00
2019-10-31 00:00:00
2019-11-07 00:00:00
2019-11-14 00:00:00
2019-11-21 00:00:00
2019-11-28 00:00:00
2019-12-05 00:00:00
2019-12-12 00:00:00
2019-12-19 00:00:00
2019-12-26 00:00:00


{datetime.datetime(2019, 1, 17, 0, 0): 199792.2236328125,
 datetime.datetime(2019, 1, 24, 0, 0): 199758.89385986328,
 datetime.datetime(2019, 1, 31, 0, 0): 199913.8201599121,
 datetime.datetime(2019, 2, 7, 0, 0): 199873.51445007324,
 datetime.datetime(2019, 2, 14, 0, 0): 200029.61376953125,
 datetime.datetime(2019, 2, 21, 0, 0): 200165.16217041016,
 datetime.datetime(2019, 2, 28, 0, 0): 200193.33294677734,
 datetime.datetime(2019, 3, 7, 0, 0): 200059.63314819336,
 datetime.datetime(2019, 3, 14, 0, 0): 200050.8589630127,
 datetime.datetime(2019, 3, 21, 0, 0): 200039.77459716797,
 datetime.datetime(2019, 3, 28, 0, 0): 200040.92892456055,
 datetime.datetime(2019, 4, 4, 0, 0): 200320.56950378418,
 datetime.datetime(2019, 4, 11, 0, 0): 200417.55436706543,
 datetime.datetime(2019, 4, 18, 0, 0): 200511.07624816895,
 datetime.datetime(2019, 4, 25, 0, 0): 200328.65055847168,
 datetime.datetime(2019, 5, 2, 0, 0): 204710.5064974594,
 datetime.datetime(2019, 5, 9, 0, 0): 204407.42628009795,
 datet

In [45]:
# Plot the portfolio value over time
fig = go.Figure()
fig.add_trace(go.Scatter(x=list(agg_values_vF_rational.keys()), y=list(agg_values_vF_rational.values()),
                    mode='lines',
                    name='lines'))
fig.show()

In [46]:
# Plot the rational vs regular portfolios over time
fig = go.Figure()
fig.add_trace(go.Scatter(x=list(agg_values_vF.keys()), y=list(agg_values_vF.values()),
                    mode='lines',
                    name='Actual'))
fig.add_trace(go.Scatter(x=list(agg_values_vF_rational.keys()), y=list(agg_values_vF_rational.values()),
                    mode='lines',
                    name='Rational Execution Times'))
fig.show()

In [47]:
transactions_rational_2 = pd.read_excel('../data/Investment_Transaction_Detail (1) copy3.xlsx')
transactions_rational_2.head()

trades_rational2 = []
for i in range(len(transactions_rational_2)):
    trade = Trade(ticker=transactions_rational_2['Ticker'][i],
                  date=transactions_rational_2['Date'][i],
                  qty=transactions_rational_2['Quantity'][i],
                  price=transactions_rational_2['Value'][i] / transactions_rational_2['Quantity'][i])
    trades_rational2.append(trade)

pb_rational2 = PortfolioBuilder(trades_rational2, backup_price_data=backup_data)
transactions_rational_2 = pb_rational2.get_transactions()
transactions_rational_2.head()

,ticker,qty,price,date
0,CASH,200000.00,1.00,2019-01-09
0,PH,25.00,157.98,2019-01-09
0,CASH,-3874.50,1.00,2019-01-09
0,NTDOY,725.00,8.61,2019-04-26
0,CASH,-6242.25,1.00,2019-04-26


In [48]:
# get weekly port values over the last year
agg_values_vF_rational2 = {}
for i in range(1, (53 * 4)):
    dt = datetime.datetime(2019, 1, 10) + datetime.timedelta(days=i*7)
    print(dt)
    dated_portfolio = pb_rational2.get_portfolio(up_to=dt)
    agg_val = dated_portfolio['Value'].sum()
    agg_values_vF_rational2[dt] = agg_val

agg_values_vF_rational2

2019-01-17 00:00:00
2019-01-24 00:00:00
2019-01-31 00:00:00
2019-02-07 00:00:00
2019-02-14 00:00:00
2019-02-21 00:00:00
2019-02-28 00:00:00
2019-03-07 00:00:00
2019-03-14 00:00:00
2019-03-21 00:00:00
2019-03-28 00:00:00
2019-04-04 00:00:00
2019-04-11 00:00:00
2019-04-18 00:00:00
2019-04-25 00:00:00
2019-05-02 00:00:00
2019-05-09 00:00:00
2019-05-16 00:00:00
2019-05-23 00:00:00
2019-05-30 00:00:00
2019-06-06 00:00:00
2019-06-13 00:00:00
2019-06-20 00:00:00
2019-06-27 00:00:00
2019-07-04 00:00:00
2019-07-11 00:00:00
2019-07-18 00:00:00
2019-07-25 00:00:00
2019-08-01 00:00:00
2019-08-08 00:00:00
2019-08-15 00:00:00
2019-08-22 00:00:00
2019-08-29 00:00:00
2019-09-05 00:00:00
2019-09-12 00:00:00
2019-09-19 00:00:00
2019-09-26 00:00:00
2019-10-03 00:00:00
2019-10-10 00:00:00
2019-10-17 00:00:00
2019-10-24 00:00:00
2019-10-31 00:00:00
2019-11-07 00:00:00
2019-11-14 00:00:00
2019-11-21 00:00:00
2019-11-28 00:00:00
2019-12-05 00:00:00
2019-12-12 00:00:00
2019-12-19 00:00:00
2019-12-26 00:00:00


{datetime.datetime(2019, 1, 17, 0, 0): 199792.22439575195,
 datetime.datetime(2019, 1, 24, 0, 0): 199758.894241333,
 datetime.datetime(2019, 1, 31, 0, 0): 199913.81939697266,
 datetime.datetime(2019, 2, 7, 0, 0): 199873.5136871338,
 datetime.datetime(2019, 2, 14, 0, 0): 200029.6145324707,
 datetime.datetime(2019, 2, 21, 0, 0): 200165.16217041016,
 datetime.datetime(2019, 2, 28, 0, 0): 200193.33409118652,
 datetime.datetime(2019, 3, 7, 0, 0): 200059.63314819336,
 datetime.datetime(2019, 3, 14, 0, 0): 200050.85858154297,
 datetime.datetime(2019, 3, 21, 0, 0): 200039.77459716797,
 datetime.datetime(2019, 3, 28, 0, 0): 200040.92930603027,
 datetime.datetime(2019, 4, 4, 0, 0): 200320.56874084473,
 datetime.datetime(2019, 4, 11, 0, 0): 200417.55474853516,
 datetime.datetime(2019, 4, 18, 0, 0): 200511.07586669922,
 datetime.datetime(2019, 4, 25, 0, 0): 200328.65132141113,
 datetime.datetime(2019, 5, 2, 0, 0): 204710.5064974594,
 datetime.datetime(2019, 5, 9, 0, 0): 204407.42589862822,
 dateti

In [55]:
# Plot all three portfolios over time
fig = go.Figure()
fig.add_trace(go.Scatter(x=list(agg_values_vF.keys()), y=list(agg_values_vF.values()),
                    mode='lines',
                    name='Actual Portfolio'))
fig.add_trace(go.Scatter(x=list(agg_values_vF_rational.keys()), y=list(agg_values_vF_rational.values()),
                    mode='lines',
                    name='Rational Execution Times'))
fig.add_trace(go.Scatter(x=list(agg_values_vF_rational2.keys()), y=list(agg_values_vF_rational2.values()),
                    mode='lines',
                    name='Rational Execution Times & Trades'))
fig.update_layout(title_text="Theoretical Portfolio Value Over Time", xaxis_title="Date", yaxis_title="Portfolio Value")
# Move the legend so it takes less space
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))
# change the font to garamond
fig.update_layout(font_family="Courier New, monospace")
fig.show()